In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Multicampus Semi-Project/data/서울시 비상소화장치 위치정보.csv', encoding='CP949')

In [ ]:
from pyproj import Proj, transform

# 좌표계 정의
# 한국 중부원점(Korea 2000 Central Belt) EPSG:5186
proj_korea = Proj(init='epsg:5186')
# WGS84
proj_wgs84 = Proj(init='epsg:4326')

# X좌표와 Y좌표를 위도와 경도로 변환하는 함수
def convert_coords(x, y):
    lon, lat = transform(proj_korea, proj_wgs84, x, y)
    return lat, lon

# 모든 좌표 변환
data[['위도', '경도']] = data.apply(lambda row: convert_coords(row['X좌표'], row['Y좌표']), axis=1, result_type="expand")

# 변환된 위도와 경도를 포함한 데이터의 처음 몇 줄 확인
data[['연번', '위도', '경도']].head()

/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
<ipython-input-4-5e077648cc54>:11: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = transform(proj_korea, proj_wgs84, x, y)


,연번,위도,경도
0,1,37.570543,126.914379
1,2,37.588984,126.909110
2,3,37.596040,126.951373
3,4,37.600518,126.950699
4,5,37.596021,126.904240


In [ ]:
import pandas as pd
import requests
import time

# 역지오코딩 함수
def reverse_geocode(longitude, latitude):
    url = 'https://dapi.kakao.com/v2/local/geo/coord2address.json'
    params = {'x': longitude, 'y': latitude}
    headers = {'Authorization': 'KakaoAK YOUR_REST_API_KEY'}
    response = requests.get(url, params=params, headers=headers)
    address_info = response.json()
    try:
        address = address_info['documents'][0]['address']['address_name']
    except (IndexError, KeyError):
        address = "주소를 찾을 수 없습니다"
    return address

# 도로명 주소를 담을 리스트
addresses = []

# 모든 좌표에 대해 역지오코딩 수행
for idx, row in data.iterrows():
    addr = reverse_geocode(row['경도'], row['위도'])
    addresses.append(addr)
    # 요청 간 일정 시간 간격 유지 (API 제한을 고려하여)
    time.sleep(1)

# 도로명 주소를 데이터셋에 추가
data['도로명주소'] = addresses

# 데이터 확인
print(data)

KeyboardInterrupt: 

In [ ]:
data.to_csv('/content/drive/MyDrive/Multicampus Semi-Project/data/서울시 비상소화장치.csv')

In [11]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Multicampus Semi-Project/data/서울시 비상소화장치.csv')
data

,Unnamed: 0,연번,비상소화장치일련번호,일련번호,X좌표,Y좌표,비상소화장치고유번호,사용구분,삭제여부,업데이트날짜,위도,경도,도로명주소
0,0,1,110080030,451,192436.056205,552336.954983,41,52201,N,20231116120041,37.570543,126.914379,서울 서대문구 남가좌동 295-5
1,1,2,110080031,452,191972.542887,554384.191076,54,52202,N,20231017120007,37.588984,126.909110,서울 은평구 증산동 131-111
2,2,3,110080032,453,195705.680844,555164.495574,56,52201,N,20231111120009,37.596040,126.951373,서울 서대문구 홍은동 9-249
3,3,4,110080033,454,195646.355737,555661.516783,57,52201,N,20231111120009,37.600518,126.950699,서울 서대문구 홍은동 8-1548
4,4,5,110080034,455,191543.205353,555165.589383,55,52201,N,20231024120006,37.596021,126.904240,서울 은평구 신사동 237-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2690,2690,2691,110086927,0,195867.263447,550812.724721,000118,52201,N,20231115120002,37.556831,126.953228,서울 마포구 아현동 354-49
2691,2691,2692,110086928,0,195978.501371,550812.366983,000119,52201,N,20231109120021,37.556829,126.954487,서울 마포구 아현동 354-49
2692,2692,2693,110086929,0,196051.060762,550814.241866,000120,52201,N,20231109120021,37.556846,126.955308,서울 마포구 아현동 354-49
2693,2693,2694,110086930,0,196029.387123,550750.448794,000121,52201,N,20231109120021,37.556271,126.955063,서울 마포구 아현동 354-49


In [23]:
data['삭제여부'].value_counts()

N    2695
Name: 삭제여부, dtype: int64

In [12]:
data.loc[data['도로명주소']=='주소를 찾을 수 없습니다']

,Unnamed: 0,연번,비상소화장치일련번호,일련번호,X좌표,Y좌표,비상소화장치고유번호,사용구분,삭제여부,업데이트날짜,위도,경도,도로명주소
66,66,67,110081513,361,202970.817692,554674.529114,145,52201,N,20231112120003,37.591630,127.033638,주소를 찾을 수 없습니다
640,640,641,110084532,0,202556.316049,551135.079242,000138,52201,N,20231105120007,37.559741,127.028932,주소를 찾을 수 없습니다
1680,1680,1681,110080722,998,199614.887710,554511.085325,4-52,52201,N,20231106120008,37.590163,126.995640,주소를 찾을 수 없습니다
1691,1691,1692,110080309,1318,207805.771501,540105.561105,000001,52201,N,20231113120034,37.460336,127.088229,주소를 찾을 수 없습니다


In [13]:
data.loc[66, '도로명주소'] = '서울 성북구 종암동 29-27'
data.loc[640, '도로명주소'] = '서울 성동구 행당동 325-16'
data.loc[1680, '도로명주소'] = '서울 종로구 명륜4길 22-5'
data.loc[1691, '도로명주소'] = '서울 서초구 내곡동 1-31595'

In [14]:
data.loc[66]

Unnamed: 0                  66
연번                          67
비상소화장치일련번호           110081513
일련번호                       361
X좌표              202970.817692
Y좌표              554674.529114
비상소화장치고유번호                 145
사용구분                     52201
삭제여부                         N
업데이트날짜          20231112120003
위도                    37.59163
경도                  127.033638
도로명주소         서울 성북구 종암동 29-27
Name: 66, dtype: object

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2695 entries, 0 to 2694
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  2695 non-null   int64  
 1   연번          2695 non-null   int64  
 2   비상소화장치일련번호  2695 non-null   int64  
 3   일련번호        2695 non-null   int64  
 4   X좌표         2695 non-null   float64
 5   Y좌표         2695 non-null   float64
 6   비상소화장치고유번호  2695 non-null   object 
 7   사용구분        2695 non-null   int64  
 8   삭제여부        2695 non-null   object 
 9   업데이트날짜      2695 non-null   int64  
 10  위도          2695 non-null   float64
 11  경도          2695 non-null   float64
 12  도로명주소       2695 non-null   object 
dtypes: float64(4), int64(6), object(3)
memory usage: 273.8+ KB


In [16]:
import folium

# 서울시 중심부의 위도와 경도
seoul_center = [37.5665, 126.9780]

# 서울시 지도 생성
seoul_map = folium.Map(location=seoul_center, zoom_start=11)

# 데이터의 일부만 사용하여 마커 추가
for idx, row in data.iterrows():
    folium.Marker([row['위도'], row['경도']], popup=f'도로명주소: {row["도로명주소"]}').add_to(seoul_map)

seoul_map

Output hidden; open in https://colab.research.google.com to view.

In [17]:
import folium
from folium.plugins import MarkerCluster

# 서울시 중심부의 위도와 경도
seoul_center = [37.5665, 126.9780]

# 서울시 지도 생성
seoul_map = folium.Map(location=seoul_center, zoom_start=10)

# 마커 클러스터 생성
marker_cluster = MarkerCluster().add_to(seoul_map)

# 데이터의 모든 행에 대해 마커 추가 및 클러스터에 추가
for idx, row in data.iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=f'도로명주소: {row["도로명주소"]}',
    ).add_to(marker_cluster)

seoul_map

Output hidden; open in https://colab.research.google.com to view.

In [25]:
data['자치구'] = data['도로명주소'].str.split().str[1]
data['자치구'].value_counts()

종로구     267
중구      220
성북구     219
동작구     175
관악구     169
용산구     149
서대문구    140
마포구     123
강북구     105
성동구     105
동대문구    103
중랑구      96
은평구      89
강동구      83
영등포구     82
강남구      79
구로구      72
서초구      71
광진구      64
도봉구      61
노원구      53
강서구      52
양천구      51
송파구      34
금천구      33
Name: 자치구, dtype: int64

In [26]:
data.to_csv('/content/drive/MyDrive/Multicampus Semi-Project/data/서울시 비상소화장치 위치.csv')

In [ ]:
# 서울시 지도 생성
seoul_map = folium.Map(location=seoul_center, zoom_start=11)

# 구별로 클러스터 생성
for 자치구 in data['자치구'].unique():
    cluster = MarkerCluster(name=자치구).add_to(seoul_map)
    자치구_data = data[data['자치구'] == 자치구]

    for idx, row in 자치구_data.iterrows():
        folium.Marker(
            location=[row['위도'], row['경도']],
            popup=f'도로명주소: {row["도로명주소"]}',
        ).add_to(cluster)

folium.LayerControl().add_to(seoul_map)  # 레이어 컨트롤 추가 (선택적)

# 지도 저장
seoul_map

Output hidden; open in https://colab.research.google.com to view.

In [29]:
data.head()

,Unnamed: 0,연번,비상소화장치일련번호,일련번호,X좌표,Y좌표,비상소화장치고유번호,사용구분,삭제여부,업데이트날짜,위도,경도,도로명주소,자치구
0,0,1,110080030,451,192436.056205,552336.954983,41,52201,N,20231116120041,37.570543,126.914379,서울 서대문구 남가좌동 295-5,서대문구
1,1,2,110080031,452,191972.542887,554384.191076,54,52202,N,20231017120007,37.588984,126.909110,서울 은평구 증산동 131-111,은평구
2,2,3,110080032,453,195705.680844,555164.495574,56,52201,N,20231111120009,37.596040,126.951373,서울 서대문구 홍은동 9-249,서대문구
3,3,4,110080033,454,195646.355737,555661.516783,57,52201,N,20231111120009,37.600518,126.950699,서울 서대문구 홍은동 8-1548,서대문구
4,4,5,110080034,455,191543.205353,555165.589383,55,52201,N,20231024120006,37.596021,126.904240,서울 은평구 신사동 237-1,은평구


In [32]:
import folium
from folium.plugins import MarkerCluster

# 데이터 로드 함수
def load_data():
    data = pd.read_csv('/content/drive/MyDrive/Multicampus Semi-Project/data/서울시 비상소화장치 위치.csv')
    return data

# 지도 생성 함수
def create_seoul_map(data, seoul_center, zoom_start=11):
    seoul_map = folium.Map(location=seoul_center, zoom_start=zoom_start)

    # 구별로 클러스터 생성
    for 자치구 in data['자치구'].unique():
        cluster = MarkerCluster(name=자치구).add_to(seoul_map)
        자치구_data = data[data['자치구'] == 자치구]

        for idx, row in 자치구_data.iterrows():
            folium.Marker(
                location=[row['위도'], row['경도']],
                popup=f'도로명주소: {row["도로명주소"]}',
            ).add_to(cluster)

    # 레이어 컨트롤 추가
    folium.LayerControl().add_to(seoul_map)

    return seoul_map

def main():
    # 데이터 로드
    data = load_data()

    # 서울시 중심 좌표
    seoul_center = [37.5665, 126.9780]

    # 서울시 지도 생성 및 표시
    seoul_map = create_seoul_map(data, seoul_center)

    # seoul_map.save('seoul_map.html')
    display(seoul_map)

if __name__ == "__main__":
    main()